In [1]:
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import curve_fit

In [60]:
def generate_Hamiltonian(n, omegas, Js):
    """
    Generates the Hamiltonian for a system with given frequencies and couplings.

    Parameters:
    omegas (list): List of qubit frequencies.
    Js (np.ndarray): Array of coupling strengths, length 2n-3. First n-1 elements are J (i, i+1), next n-2 are J_parallel (i, i+2).

    Returns:
    np.ndarray: Hamiltonian matrix.
    """
    n = len(omegas)
    H = np.zeros((n, n), dtype=complex)

    J = Js[:n-1]
    J_parallel = Js[n-1:]

    for i in range(n):
        H[i, i] = omegas[i]
        if i < n - 1:
            H[i, i + 1] = J[i]
            H[i + 1, i] = np.conjugate(J[i])
        if i < n - 2:
            H[i, i + 2] = J_parallel[i]
            H[i + 2, i] = np.conjugate(J_parallel[i])


    print(f'Js; {Js}')
    

    return H

In [119]:
def cost_function(omegas, Js, target_populations, eigenvector_index=None):
    """
    Computes the cost as the norm of the difference between the computed and provided eigenvectors.

    Parameters:
    omegas (list): List of qubit frequencies.
    Js (np.ndarray): Array of coupling strengths, length 2n-3. First n-1 elements are J (i, i+1), next n-2 are J_parallel (i, i+2).
    target_populations (np.ndarray): Provided populations to compare against.
    eigenvector_index (int): Index of the eigenvector to compute.

    Returns:
    float: Cost value (difference norm).
    """
    H = generate_Hamiltonian(len(omegas), omegas, Js)
    eigenvalues, eigenvectors = np.linalg.eigh(H)

    # Sort eigenvalues and eigenvectors
    idx = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]

    if eigenvector_index is not None and isinstance(eigenvector_index, int):
        computed_eigenvector = eigenvectors[:, eigenvector_index]
        computed_eigenvector = computed_eigenvector / np.linalg.norm(computed_eigenvector)
        computed_populations = np.abs(computed_eigenvector)**2
        cost = np.linalg.norm(computed_populations - target_populations)
        print(f'computed_populations: {computed_populations}')
        print(f'cost: {cost}')
        return cost
    else:
        # target_populations is assumed to be (n, n), compare each column of eigenvectors
        costs = []
        for idx in range(eigenvectors.shape[1]):
            computed_eigenvector = eigenvectors[:, idx]
            computed_eigenvector = computed_eigenvector / np.linalg.norm(computed_eigenvector)
            computed_populations = np.abs(computed_eigenvector)**2
            cost = np.linalg.norm(computed_populations - target_populations[:, idx])
            print(f'computed_populations (idx={idx}): {computed_populations}')
            print(f'cost (idx={idx}): {cost}')
            costs.append(cost)
        return np.sum(np.array(costs))

In [120]:
from scipy.optimize import minimize

def optimize_hamiltonian_params(initial_omegas, initial_Js, target_populations, eigenvector_index):
    """
    Optimizes Hamiltonian parameters to match the target populations at the specified eigenvector index.

    Parameters:
    initial_omegas (list): Initial guess for qubit frequencies.
    initial_Js (np.ndarray): Initial guess for coupling strengths
    target_populations (np.ndarray): Target populations to match.
    eigenvector_index (int): Index of the eigenvector to optimize.

    Returns:
    dict: Optimized parameters.
    """

    n = len(initial_omegas)


    def objective(params):
        omegas = params[:n]
        Js = params[n:]
        return cost_function(omegas, Js, target_populations, eigenvector_index)

    initial_params = np.array(list(initial_omegas) + list(initial_Js))
    result = minimize(objective, initial_params, method='Nelder-Mead')
    optimized_omegas = result.x[:n]
    optimized_Js = result.x[n:]

    optimized_J_perps = optimized_Js[:n - 1]
    optimized_J_parallels = optimized_Js[n - 1:]


    optimized_H = generate_Hamiltonian(n, optimized_omegas, optimized_Js)

    return {
        'omegas': optimized_omegas,
        'J': optimized_J_perps,
        'J_parallel': optimized_J_parallels,
        'success': result.success,
        'cost': result.fun,
        'Hamiltonian': optimized_H
    }

In [131]:
# Example usage of optimize_hamiltonian_params

# Initial guesses for parameters
initial_omegas = [0,0,0]
initial_Js = [-5, -5, -7]

# Target populations (for example, we want the populations to be [0.2, 0.5, 0.3])
# target_populations = np.array([0.356, 0.190, 0.356]) # index 0
# target_populations = np.array([0.369, 0.156, 0.361]) # index 1
# target_populations = np.array([0.053, 0.624, 0.303]) # index 2

# Construct the target populations as a 3x3 matrix, each column for an eigenvector index
target_populations = np.array([
    [0.356, 0.190, 0.356],
    [0.369, 0.156, 0.361],
    [0.053, 0.624, 0.303]
])


# Index of the eigenvector to match (e.g., ground state: 0)
eigenvector_index = 0


if eigenvector_index is not None and isinstance(eigenvector_index, int):
    target_populations = target_populations[eigenvector_index,:]


target_populations = np.array([0.715, 0.063, 0.125]) # index None



print(f'target_populations: {target_populations}')



# Run optimization
print(np.sum(target_populations))
target_populations = target_populations / np.sum(target_populations)


result = optimize_hamiltonian_params(
    initial_omegas,
    initial_Js,
    target_populations,
    eigenvector_index
)

# print("Optimized Parameters:", result)

for key, value in result.items():
    if isinstance(value, np.ndarray):
        print(f"{key}: {value.tolist()}")
    else:
        print(f"{key}: {value}")

target_populations: [0.715 0.063 0.125]
0.903
Js; [-5. -5. -7.]
computed_populations: [0.36090174 0.27819651 0.36090174]
cost: 0.5278401082605497
Js; [-5. -5. -7.]
computed_populations: [0.36089525 0.27819969 0.36090506]
cost: 0.527848065674901
Js; [-5. -5. -7.]
computed_populations: [0.36090493 0.27819015 0.36090493]
cost: 0.5278363396739929
Js; [-5. -5. -7.]
computed_populations: [0.36090506 0.27819969 0.36089525]
cost: 0.5278359195431775
Js; [-5.25 -5.   -7.  ]
computed_populations: [0.36298694 0.28260823 0.35440484]
cost: 0.5251975204453999
Js; [-5.   -5.25 -7.  ]
computed_populations: [0.35440484 0.28260823 0.36298694]
cost: 0.5357678065474654
Js; [-5.   -5.   -7.35]
computed_populations: [0.3652899 0.2694202 0.3652899]
cost: 0.5227265078037192
Js; [-5.08333333 -4.75       -7.11666667]
computed_populations: [0.36953425 0.27228196 0.35818379]
cost: 0.5173177325037005
Js; [-5.125 -4.5   -7.175]
computed_populations: [0.37705264 0.26707798 0.35586938]
cost: 0.5081653607285468
Js; [-5

In [132]:
optimized_hamiltonian = result['Hamiltonian']

eigenvalues, eigenvectors = np.linalg.eigh(optimized_hamiltonian)

print('optimized hamiltonian:\n', np.round(optimized_hamiltonian.real, 2))
print(f'eigenvalues: {eigenvalues}\n')
print(f'eigenvectors:\n {eigenvectors.real}\n')
print(f'all populations:\n {np.abs(eigenvectors)**2}\n')



# normalized_eigenvectors = eigenvectors / np.linalg.norm(eigenvectors, axis=0)

if eigenvector_index is not None and isinstance(eigenvector_index, int):
    populations = np.abs(eigenvectors[:,eigenvector_index])**2
else:
    # target_populations is assumed to be (n, n), compare each column of eigenvectors
    populations = np.abs(eigenvectors)**2

print('populations')
print(populations)

print('target populations')
print(target_populations)

print("Cost:", result['cost'])

print(np.linalg.norm(target_populations - populations, ord=1))

optimized hamiltonian:
 [[ -0. -13. -13.]
 [-13.   0.  13.]
 [-13.  13.   0.]]
eigenvalues: [-12.99777684 -12.99773651  25.99764386]

eigenvectors:
 [[ 0.81574487 -0.03551209  0.57732071]
 [ 0.37709355 -0.7241829  -0.57737299]
 [ 0.43858951  0.68869298 -0.57735711]]

all populations:
 [[0.66543969 0.00126111 0.3332992 ]
 [0.14219955 0.52444088 0.33335957]
 [0.19236076 0.47429802 0.33334123]]

populations
[0.66543969 0.14219955 0.19236076]
target populations
[0.79180509 0.06976744 0.13842746]
Cost: 0.15531717457629132
0.25273080096561634
